## Target- urgent withdrawn by nurse(within 24hr)
* shift_applications status == 'cancelled' + shift_application prevStatus == 'confirmed'


### IDs -> only look at shifts application, leave each shifts after
* shift_applications id
* shift_applications user_id
* shift_applications shift_id


### Numeric Field
* shifts rate
* shifts net_pay: rate - nursedash profit
* shift_applications distance
* shift break time
* user withdrawn times: 
    Problem: 
        1. We don't know in reality that you will withdraw your previous one.
        2. cannot be used to predict new user
* user applied times:
* <b>prev SA/CW rate<b/>


## Time
### shift, shift app time
* S_Create (shift created) => SA_Create (application created) => CWTime (comfim withdrawn) => Start_Time (shift start)
* S_Create: shift create time
* SA_Create: shift application create time
* CWTime: shift withdrawn time, whole confirmed withdrawn population (include within 24hr and > 24hr)
* Start_Time: shift start time
##### calculated field, graph in hours
* <b>S_Create2SA_Create<b/>: shift application create time - shift create time
* <b>S_Create2Start_Time<b/>: shift start time - shift create time
        1. has negative values. delete maybe?
* <b>SA_Create2Start_Time<b/>: shift start time - shift application create time
        1. has negative values. delete maybe?

### User
* user approvedAt:
* user createdAt:
#### Calculated field
* U_Create2U_approved: User approve time - User create time
* <b>U_Create2now<b/>: now - User create time, in months(seconds/2629746)
* <b>U_Approve2now<b/>: now - User approve time


### Categorical
* shifts role: position name + type
* facilities name = facilities short_name
* facilities areaId = facilities areaName
* withdrawnInfo_key, withdrawnInfo_value
* facilities segmentName: Senior Living = 1, Healthcare = 0
* Users enableNotifications:
* Users emailNotifications:
* Users appNotifications:
* Users allowedNotifications:


### ?
* facilities allowedQualification
* facilities createdAt
* facilities rates
* nurse shift withdrawn by admin, when policy start, how many withdrawn by admin
* shifts = unit
* role
* users updateAt ?
* users relationToFacility

# Concat data

In [128]:
import pandas as pd
import numpy as np

In [129]:
df1 = pd.read_csv('0-2 new.csv')
df2 = pd.read_csv('2-5.csv')
df3 = pd.read_csv('5-9.csv')
df4 = pd.read_csv('9-12.csv')
df5 = pd.read_csv('12-18.csv')

main = pd.concat([df1, df2, df3, df4, df5], axis = 0)

main.to_csv('main.csv')

# Data Cleaning

In [130]:
df = pd.read_csv('main.csv').drop(columns = ['Unnamed: 0'])
df.columns

Index(['id', 'user_id', 'shift_id', 'status', 'createdAt',
       'hasNurseCheckEvent', 'hasBreakTime', 'prevStatus', 'distance', 'id.1',
       'facility_id', 'description', 'start_time', 'assigned_nurse_id', 'rate',
       'net_pay', 'unit', 'type', 'createdAt.1', 'qualifications', 'breakTime',
       'prevStatus.1', 'id.2', 'name', 'short_name', 'createdAt.2', 'rates',
       'S_Create', 'SA_Create', 'S_create2SA_Create', 'Start_Time',
       'SA_Create2Start_Time', 'S_Create2Start_Time', 'CW_Time',
       'CW_Time2Start_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time',
       'allowedQualifications', 'areaId', 'areaName', 'segmentName',
       'U_create2now', 'U_approve2now', 'U_create2U_approve',
       'withdrawnInfo_key', 'withdrawnInfo_value'],
      dtype='object')

In [131]:
# fill these column with 0, no cw make these column nan
df[['CW_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time']] = df[['CW_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time']].fillna(0)

# make time to datetime object
df['start_time'] = pd.to_datetime(df['start_time'])
df['createdAt'] = pd.to_datetime(df['createdAt'])

In [132]:
df.shape

(172051, 46)

In [133]:
#df['tar'] = df.apply(lambda x: 1 if x['status'] == 'withdrawn' and x['prevStatus'] == 'confirmed' and x['withdrawnInfo_value'] == 'nurse' and df['CW_Time2Start_Time'] > -24 else 0, axis =1)

In [134]:
# There are some negative numbers in these column. Target all = 0, drop for now
df = df[(df["S_Create2Start_Time"]>0) & (df["SA_Create2Start_Time"]>0)]

# sort by time that shift was created
df = df.sort_values(by = 'createdAt').reset_index(drop = True)

# Feature Engineering

### Target: urgent withdrawn as 1

In [135]:
def CW_by_nurse(row):
    if row['status']=='withdrawn' and row['prevStatus'] == 'confirmed':
        if row['withdrawnInfo_value'] == 'nurse':
            if row['CW_Time2Start_Time'] < 0 and row['CW_Time2Start_Time'] >= -24:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0
    
df['target'] = df.apply (lambda row: CW_by_nurse(row), axis=1)

In [136]:
df['target'].value_counts()

0    165745
1      4372
Name: target, dtype: int64

### Count previsous shift application

In [137]:
# track by using dictionary
count_prev_SA = []
test = df[['user_id', 'createdAt']]

# create SA dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
sa_dict = {} 
for uid in uid_library:
    sa_dict.update({uid: 0}) 
    
for i, v in enumerate(test['user_id']):
    sa_dict[v] += 1
    count_prev_SA.append(sa_dict[v]-1)
 
df['count_prev_SA'] = count_prev_SA

### Count previsous urgent withdrawns

In [138]:
# track by using dictionary
count_prev_CW = []
test = df[['user_id', 'count_prev_SA', 'createdAt', 'target']]

# create cw dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
cw_dict = {} 
for uid in uid_library:
    cw_dict.update({uid: 0}) 
    
# fill dictionary and fill cw count
for i, v in enumerate(test['user_id']):
    if test['target'][i] == 1:
        cw_dict[v] += 1
        count_prev_CW.append(cw_dict[v]-1)
    else:
        count_prev_CW.append(cw_dict[v])
        
df['count_prev_CW'] = count_prev_CW

### Count previsous urgent withdrawns/Count previsous shift applications

In [139]:
df['prev_CW/SA_rate'] = df['count_prev_CW']/df['count_prev_SA']

# fill nan with 0, happend bc 0/0, meaning rate = 0
df['prev_CW/SA_rate'].fillna(0,inplace=True)

### Type dummy: RN = 1, LVN + LPN = 2, rest = 0


In [140]:
def create_type_dummy(row):
    if row['type']=='RN':
        return 2
    elif row['type'] == 'LVN' or row['type'] == 'LPN':
        return 1
    else:
        return 0
    
df['type_d'] = df.apply(lambda row: create_type_dummy(row), axis=1)

### Type dummy: Senior Living, Healthcare

In [141]:
def create_segmentName_dummy(row):
    if row['segmentName']=='Senior Living':
        return 1
    else:
        return 0
    
df['segmentName_d'] = df.apply(lambda row: create_segmentName_dummy(row), axis=1)

### <font color = green>output feature dataframe

In [142]:
feature_df = df[['prev_CW/SA_rate','S_create2SA_Create','S_Create2Start_Time','SA_Create2Start_Time','U_create2now',
                 'U_approve2now', 'type_d','segmentName_d','net_pay','target', 'createdAt', 'start_time']]
feature_df.to_csv('model_data.csv')

In [109]:
# def profit_calculation(model, x_value, y_value):
#     d_cutoff = 1/2

#     decision = list(model.predict_proba(x_value)[:,1] < d_cutoff)   
#     decision = list(model.predict(x_value))
#     y = list(y_value)
#     n_obs = len(y)

#     cum_profit = 0

#     for i in range(n_obs):
#         if decision[i] == True and y[i] == 1: 
#             profit = 0 
#         elif decision[i] == True and y[i] == 0: 
#             profit = 8 
#         elif decision[i] == False and y[i] == 1: 
#             profit = -2
#         else:
#             profit = 10 
#         cum_profit = cum_profit + profit 

#     average_net_profit = cum_profit / n_obs 
#     return average_net_profit